In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import cv2
from PIL import Image

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras import layers, models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/archive.zip'

In [ ]:
# train_path='../input/gender-classification-dataset/Training'
# test_path='../input/gender-classification-dataset/Validation'

# male_train='../input/gender-classification-dataset/Training/male'
# female_train='../input/gender-classification-dataset/Training/female'

# male_test='../input/gender-classification-dataset/Validation/male'
# female_test='../input/gender-classification-dataset/Validation/female'

train_path='/content/Training'
test_path='/content/Validation'

male_train='/content/Training/male'
female_train='/content/Training/female'

male_test='/content/Validation/male'
female_test='/content/Validation/female'

In [ ]:
category=os.listdir(train_path)
category

In [ ]:
num_category=len(category)
num_category

In [ ]:
train_images=[]
for cat in category:
    folder=train_path+"/"+cat
    train_images.append(len(os.listdir(folder)))

In [ ]:
train_images

In [ ]:
plt.bar(category, train_images)

In [ ]:
category=os.listdir(test_path)
num_category=len(category)
print(num_category)

In [ ]:
test_images=[]
for cat in category:
    folder=test_path+'/'+cat
    test_images.append(len(os.listdir(folder)))

In [ ]:
plt.bar(category, test_images)

In [ ]:
test_images

In [ ]:
Image.open(train_path+'/female/131489.jpg.jpg')

In [ ]:
Image.open(train_path+'/male/090646.jpg.jpg')

In [ ]:
fem_train=[]
label1=[]
for i in os.listdir(female_train):
    if os.path.isfile(female_train+'/'+i):
        female=Image.open(female_train+'/'+i)
        female=female.resize((50,50), Image.ANTIALIAS).convert("L")
        female=np.asarray(female)/255.0
        fem_train.append(female)
        label1.append(0)

In [ ]:
m_train=[]
for i in os.listdir(male_train):
    if os.path.isfile(male_train+'/'+i):
        male=Image.open(male_train+'/'+i)
        male=male.resize((50,50), Image.ANTIALIAS).convert("L")
        male=np.asarray(male)/255.0
        m_train.append(male)
        label1.append(1)

In [ ]:
x_train=np.concatenate((fem_train, m_train), axis=0)
y_train=np.asarray(label1)
y_train=y_train.reshape(x_train.shape[0], 1)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
fem_test=[]
label2=[]
for i in os.listdir(female_test):
    if os.path.isfile(female_test+'/'+i):
        female=Image.open(female_test+'/'+i)
        female=female.resize((50,50), Image.ANTIALIAS).convert("L")
        female=np.asarray(female)/255.0
        fem_test.append(female)
        label2.append(0)

In [ ]:
m_test=[]
for i in os.listdir(male_test):
    if os.path.isfile(male_test+'/'+i):
        male=Image.open(male_test+'/'+i)
        male=male.resize((50,50), Image.ANTIALIAS).convert("L")
        male=np.asarray(male)/255.0
        m_test.append(male)
        label2.append(1)

In [ ]:
x_test=np.concatenate((fem_test, m_test), axis=0)
y_test=np.asarray(label2)
y_test=y_test.reshape(x_test.shape[0],1)

In [ ]:
x_test.shape

In [ ]:
y_test.shape

In [ ]:
x=np.concatenate((x_train,x_test),axis=0)
y=np.concatenate((y_train,y_test),axis=0)
x=x.reshape(x.shape[0],x.shape[1]*x.shape[2])

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain, xtest, ytrain, ytest=train_test_split(x,y,random_state=2, test_size=0.2)

In [ ]:
xtr=xtrain.T
xts=xtest.T
ytr=ytrain.T
yts=ytest.T

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
xtrain.shape

In [ ]:
ytrain.shape

In [ ]:
xtest.shape

In [ ]:
ytest.shape

In [ ]:
lr = Sequential()

lr.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (None, 2500)))
lr.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
lr.add(MaxPool2D(pool_size=(2,2)))
lr.add(Dropout(0.25))


lr.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
lr.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
lr.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
lr.add(Dropout(0.25))

lr.add(Flatten())
lr.add(Dense(256, activation = "relu"))
lr.add(Dropout(0.5))
lr.add(Dense(2, activation = "softmax"))

In [ ]:
lr.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
lr.fit(xtrain, ytrain, batch_size=128, epochs=10, validation_data=(xtest, ytest))

In [ ]:
print("accuracy_score", accuracy_score(lr.predict(xtrain), ytrain))
print("accuracy_score", accuracy_score(lr.predict(xtest), ytest))

In [ ]:
img_path='/content/Woman-Face-PNG-Free-Image.png'
female=Image.open(img_path)
plt.imshow(female)
female=female.resize((50,50), Image.ANTIALIAS).convert("L")
female=np.asarray(female)/255.0
female=female.reshape(female.shape[0]*female.shape[1])
female = female.reshape(1, -1)
female.shape
result=lr.predict(female)
result=np.argmax(result)

if result==0:
    print("female")
else:
    print("male")

In [ ]:
img_path='/content/ec58198c3f652de85e735d918ef85dd8.jpg'
female=Image.open(img_path)
plt.imshow(female)
female=female.resize((50,50), Image.ANTIALIAS).convert("L")
female=np.asarray(female)/255.0
female=female.reshape(female.shape[0]*female.shape[1])
female = female.reshape(1, -1)
female.shape
result=lr.predict(female)
result=np.argmax(result)

if result==0:
    print("female")
else:
    print("male")

In [ ]:
img_path='/content/images.jpg'
female=Image.open(img_path)
plt.imshow(female)
female=female.resize((50,50), Image.ANTIALIAS).convert("L")
female=np.asarray(female)/255.0
female=female.reshape(female.shape[0]*female.shape[1])
female = female.reshape(1, -1)
female.shape
result=lr.predict(female)
result=np.argmax(result)

if result==0:
    print("female")
else:
    print("male")

In [ ]:
img_path='/content/960x0.jpg'
female=Image.open(img_path)
plt.imshow(female)
female=female.resize((50,50), Image.ANTIALIAS).convert("L")
female=np.asarray(female)/255.0
female=female.reshape(female.shape[0]*female.shape[1])
female = female.reshape(1, -1)
female.shape
result=lr.predict(female)
result=np.argmax(result)

if result==0:
    print("female")
else:
    print("male")

In [ ]:
img_path='/content/Screenshot 2022-11-13 093903.png'
female=Image.open(img_path)
plt.imshow(female)
female=female.resize((50,50), Image.ANTIALIAS).convert("L")
female=np.asarray(female)/255.0
female=female.reshape(female.shape[0]*female.shape[1])
female = female.reshape(1, -1)
female.shape
result=lr.predict(female)
result=np.argmax(result)

if result==0:
    print("female")
else:
    print("male")

In [ ]:
img_path='/content/ezgif.com-gif-maker.jpg'
female=Image.open(img_path)
plt.imshow(female)
female=female.resize((50,50), Image.ANTIALIAS).convert("L")
female=np.asarray(female)/255.0
female=female.reshape(female.shape[0]*female.shape[1])
female = female.reshape(1, -1)
female.shape
result=lr.predict(female)
result=np.argmax(result)

if result==0:
    print("female")
else:
    print("male")

In [ ]:
img_path='/content/Woman-Face-PNG-Free-Image.png'
female=Image.open(img_path)
plt.imshow(female)
female=female.resize((50,50), Image.ANTIALIAS).convert("L")
female=np.asarray(female)/255.0
female=female.reshape(female.shape[0]*female.shape[1])
female = female.reshape(1, -1)
female.shape
result=lr.predict(female)
result=np.argmax(result)

if result==0:
    print("female")
else:
    print("male")

In [ ]:
img_path='/content/images (1).jpg'
female=Image.open(img_path)
plt.imshow(female)
female=female.resize((50,50), Image.ANTIALIAS).convert("L")
female=np.asarray(female)/255.0
female=female.reshape(female.shape[0]*female.shape[1])
female = female.reshape(1, -1)
female.shape
result=lr.predict(female)
result=np.argmax(result)

if result==0:
    print("female")
else:
    print("male")

In [ ]:
img_path='/content/photo-1619182597083-17bda72c1d56.jpg'
female=Image.open(img_path)
plt.imshow(female)
female=female.resize((50,50), Image.ANTIALIAS).convert("L")
female=np.asarray(female)/255.0
female=female.reshape(female.shape[0]*female.shape[1])
female = female.reshape(1, -1)
female.shape
result=lr.predict(female)
result=np.argmax(result)

if result==0:
    print("female")
else:
    print("male")

In [ ]:
img_path='/content/pretty-woman-face-beautiful-female-model-face-clear-skin-healthy-brown-hair-close-up-pretty-woman-face-beautiful-female-147747886.jpg'
female=Image.open(img_path)
plt.imshow(female)
female=female.resize((50,50), Image.ANTIALIAS).convert("L")
female=np.asarray(female)/255.0
female=female.reshape(female.shape[0]*female.shape[1])
female = female.reshape(1, -1)
female.shape
result=lr.predict(female)
result=np.argmax(result)

if result==0:
    print("female")
else:
    print("male")